In [310]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from collections import OrderedDict
from pyspark import SparkFiles
from tables import *


spark_session = SparkSession\
                .builder\
                .master("local")\
                .appName("Cluster Data Analysis")\
                .getOrCreate()
#               .master("spark://192.168.1.16:7077") \
#               .config("spark.submit.deployMode", "client")\
#               .config("spark.dynamicAllocation.enabled", "false")\
#               .config("spark.cores.max", "4")\
#               .config("spark.dynamicAllocation.enabled", "false")\
#               .config("spark.executor.memory", "4g")\
                
spark_context = spark_session.sparkContext
sqlContext = SQLContext(spark_session)

In [311]:
# mypath = "hdfs://192.168.1.16:9000/user/ubuntu/songs_small.csv"
mypath = "songs_small.csv"

data_frame = spark_session.read\
    .option('header', 'true')\
    .option('ignoreLeadingWhiteSpace', 'true')\
    .option('ignoreTrailingWhiteSpace', 'true')\
    .csv(mypath, inferSchema=True, multiLine=True)
#data_frame.count()

In [312]:
data_frame.take(1)

In [313]:
data_frame = data_frame.withColumn("tempo", data_frame["tempo"].cast(DoubleType()))

In [ ]:
data_frame.printSchema()

In [ ]:
data_frame.registerTempTable("songs")

# spark_session.sql("select artist_name from songs where artist_name like 'b%'").show()
# spark_session.sql("select year from songs").show()
spark_session.sql("SELECT year from songs").show()

In [ ]:
spark_session.sql("select avg(loudness), year from songs group by year order by avg(tempo) desc").show()

In [ ]:
spark_context.stop()